In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Data Exploration\n",
    "\n",
    "생성된 평가 결과 데이터 탐색 및 시각화"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import json\n",
    "import os\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "import sys\n",
    "sys.path.append('..')\n",
    "\n",
    "from src.config import OUTPUT_PATHS"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. 평가 결과 로드"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 평가 결과 파일 목록\n",
    "eval_dir = OUTPUT_PATHS['evaluations']\n",
    "eval_files = [f for f in os.listdir(eval_dir) if f.endswith('.json')]\n",
    "\n",
    "print(f\"Found {len(eval_files)} evaluation files:\")\n",
    "for f in eval_files:\n",
    "    print(f\"  - {f}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 가장 최근 파일 로드\n",
    "if eval_files:\n",
    "    latest_file = sorted(eval_files)[-1]\n",
    "    filepath = os.path.join(eval_dir, latest_file)\n",
    "    \n",
    "    with open(filepath, 'r', encoding='utf-8') as f:\n",
    "        eval_data = json.load(f)\n",
    "    \n",
    "    print(f\"\\nLoaded: {latest_file}\")\n",
    "    print(f\"Service: {eval_data.get('service_name')}\")\n",
    "    print(f\"Timestamp: {eval_data.get('timestamp')}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. 평가 점수 시각화"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if eval_files and eval_data:\n",
    "    ethics_eval = eval_data.get('ethics_evaluation', {})\n",
    "    \n",
    "    # 기준별 점수 추출\n",
    "    criteria = ['bias', 'privacy', 'transparency', 'accountability', 'safety']\n",
    "    scores = [ethics_eval.get(c, {}).get('score', 0) for c in criteria]\n",
    "    labels = ['Bias', 'Privacy', 'Transparency', 'Accountability', 'Safety']\n",
    "    \n",
    "    # 바 차트\n",
    "    plt.figure(figsize=(10, 6))\n",
    "    colors = ['#ff6b6b' if s < 5 else '#feca57' if s < 7 else '#1dd1a1' for s in scores]\n",
    "    plt.bar(labels, scores, color=colors, alpha=0.7, edgecolor='black')\n",
    "    plt.axhline(y=5, color='orange', linestyle='--', label='Medium Risk Threshold')\n",
    "    plt.axhline(y=7, color='green', linestyle='--', label='Low Risk Threshold')\n",
    "    plt.ylim(0, 10)\n",
    "    plt.ylabel('Score')\n",
    "    plt.title(f'Ethics Evaluation Scores - {eval_data.get(\"service_name\")}')\n",
    "    plt.legend()\n",
    "    plt.grid(axis='y', alpha=0.3)\n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "    \n",
    "    # 전체 점수\n",
    "    overall = ethics_eval.get('overall_score', 0)\n",
    "    print(f\"\\n📊 Overall Score: {overall}/10\")\n",
    "    print(f\"⚠️  Risk Level: {ethics_eval.get('overall_risk_level', 'N/A')}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. 레이더 차트"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import numpy as np\n",
    "\n",
    "if eval_files and eval_data:\n",
    "    # 레이더 차트 설정\n",
    "    angles = np.linspace(0, 2 * np.pi, len(labels), endpoint=False).tolist()\n",
    "    scores_plot = scores + [scores[0]]  # 닫힌 도형을 위해\n",
    "    angles += angles[:1]\n",
    "    \n",
    "    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))\n",
    "    ax.plot(angles, scores_plot, 'o-', linewidth=2, label='Current Score')\n",
    "    ax.fill(angles, scores_plot, alpha=0.25)\n",
    "    ax.set_theta_offset(np.pi / 2)\n",
    "    ax.set_theta_direction(-1)\n",
    "    ax.set_xticks(angles[:-1])\n",
    "    ax.set_xticklabels(labels)\n",
    "    ax.set_ylim(0, 10)\n",
    "    ax.set_yticks([2, 4, 6, 8, 10])\n",
    "    ax.set_yticklabels(['2', '4', '6', '8', '10'])\n",
    "    ax.grid(True)\n",
    "    ax.set_title(f'Ethics Profile - {eval_data.get(\"service_name\")}', y=1.08)\n",
    "    plt.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))\n",
    "    plt.tight_layout()\n",
    "    plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. 개선 제안 요약"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if eval_files and eval_data:\n",
    "    proposals = eval_data.get('improvement_proposals', [])\n",
    "    \n",
    "    print(f\"\\n💡 Improvement Proposals: {len(proposals)}\\n\")\n",
    "    print(\"=\"*60)\n",
    "    \n",
    "    for i, prop in enumerate(proposals, 1):\n",
    "        criterion = prop.get('criterion', 'N/A')\n",
    "        priority = prop.get('priority', 'N/A')\n",
    "        recommendation = prop.get('recommendation', 'N/A')\n",
    "        \n",
    "        priority_icon = \"🔴\" if priority == \"high\" else \"🟡\" if priority == \"medium\" else \"🟢\"\n",
    "        \n",
    "        print(f\"{i}. {priority_icon} {criterion.upper()} ({priority} priority)\")\n",
    "        print(f\"   Recommendation: {recommendation}\")\n",
    "        print()"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.10.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}